# 2023 Day 5

https://adventofcode.com/2023/day/5

https://adventofcode.com/2023/day/5/input

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numba import jit
from numba.typed import List

In [2]:
inp = open('input-05.txt').read().strip()

In [3]:
# print(inp)

In [4]:
test1 = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()
print(test1)

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4


## Part 1

In [5]:
def get_seeds(para):
    return list(map(int, para.split(':')[1].strip().split()))

def get_mapping(para):
    name = para.split()[0]
    mapping = []
    for line in para.split('\n')[1:]:
        dest0, source0, n = list(map(int, line.strip().split()))
        mapping.append((dest0, source0, n))
        # for i in range(n):
        #     mapping[source0 + i] = dest0 + i
    return name, mapping

def get_maps(text):
    paras = text.split('\n\n')
    seeds = get_seeds(paras[0])
    mappings = [get_mapping(para)[1] for para in paras[1:]]
    return seeds, mappings

def get_maps_2(text):
    paras = text.split('\n\n')
    seed_nums = get_seeds(paras[0])
    mappings = [get_mapping(para)[1] for para in paras[1:]]
    seed_gens = [
        (seed1, seed1 + nseeds)
        for (seed1, nseeds)
        in zip(seed_nums[::2], seed_nums[1::2])
    ]
    return seed_gens, mappings

@jit(nopython=True)
def apply_mapping(i, mapping):
    for (dest, source, n) in mapping:
        if source <= i < source + n:
            return dest + (i - source)
    return i

def trace_seed(seed, mappings):
    n = seed
    trace = [n]
    for mapping in mappings:
        n = apply_mapping(n, mapping)
        trace.append(n)
    return n #trace

@jit(nopython=True)
def apply_reverse_mapping(i, mapping):
    for (dest, source, n) in mapping:
        if dest <= i < dest + n:
            return source + (i - dest)
    return i

@jit(nopython=True)
def trace_loc_to_seed(loc, mappings):
    n = loc
    trace = [n]
    for mapping in mappings[::-1]:
        n = apply_reverse_mapping(n, mapping)
        trace.append(n)
    return n #trace


@jit(nopython=True)
def check_seed(candidate, seed_gens):
    for (s1, s2) in seed_gens:
        if s1 <= candidate < s2:
            return True
    return False

@jit(nopython=True)
def find_nearest_reachable_loc(seed_gens, mappings, nmax=100):
    min_mapped_loc = sorted(mappings[-1])[0][0]
    locs = range(nmax)
    for loc in locs:
        seed = trace_loc_to_seed(loc, mappings)
        if check_seed(seed, seed_gens):
            return loc
    return -1

In [6]:
def type_seed_gens(seed_gens):
    return List([List(sg) for sg in seed_gens])

def type_mappings(mappings):
    return List([
        List([
            List(m)
            for m in mapping
        ])
        for mapping in mappings
    ])

In [7]:
%%time

seeds, mappings = get_maps(test1)
mappings_typed = type_mappings(mappings)
min(trace_seed(seed, mappings_typed) for seed in seeds)

CPU times: user 856 ms, sys: 18.4 ms, total: 874 ms
Wall time: 874 ms


35

In [8]:
%%time

seeds, mappings = get_maps(inp)
mappings_typed = type_mappings(mappings)
min(trace_seed(seed, mappings_typed) for seed in seeds)

CPU times: user 4.79 ms, sys: 0 ns, total: 4.79 ms
Wall time: 4.74 ms


662197086

## Part 2

In [9]:
%%time

seed_gens, mappings = get_maps_2(test1)
seed_gens_typed = type_seed_gens(seed_gens)
mappings_typed = type_mappings(mappings)
find_nearest_reachable_loc(seed_gens_typed, mappings_typed)

CPU times: user 3.36 s, sys: 118 ms, total: 3.48 s
Wall time: 3.48 s


46

In [10]:
%%time

seed_gens, mappings = get_maps_2(inp)
seed_gens_typed = type_seed_gens(seed_gens)
mappings_typed = type_mappings(mappings)
# find_nearest_reachable_loc(seed_gens_typed, mappings_typed, nmax=10000000)
find_nearest_reachable_loc(seed_gens_typed, mappings_typed, nmax=662197086)

CPU times: user 58.6 s, sys: 104 ms, total: 58.7 s
Wall time: 58.3 s


52510809